# Get relavant documents

1. load documents
2. split documents using a character limit
3. assign a unique id to all chunks
4. for each chunk make linked list prev-next
5. save to index db

1. using open ask chatbot to list file names if any if the prompt is particularly asking
2. if there are file names mentioned use them in __filter__ while getting __retriever__ from __DB__
3. Use __MultiQueryRetriever__ to generate 5 perspectives of prompt
4. use __EmbeddingsRedundantFilter__,__EmbeddingsFilter__,__DocumentCompressorPipeline__,__ContextualCompressionRetriever__ to retrieve the relavant chunks by chunking into smaller chunks and retrieve __chunks__ which having more __relavance score__
5. ```python
    	splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=10)
        redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
        relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76,k=15)
        pipeline_compressor = DocumentCompressorPipeline(transformers=[splitter, redundant_filter, relevant_filter])
        compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

    ```
6. use retriever and get __relavant children chunks__
7. get parent chunks by removing duplicate parents
8. if the answer does not specify more we can ask the user to retrieve the next set.

__Load environment variables__

In [1]:
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv("C:/fileanalyst_ai.env")

OPEN_AI_KEY =  os.getenv("OPENAI_API_KEY",default=None)

In [2]:
from langchain.document_loaders import UnstructuredWordDocumentLoader,UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQAWithSourcesChain,ConversationalRetrievalChain
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from IPython.display import display, Markdown, Latex
from langchain.callbacks import get_openai_callback

from glob import glob
import uuid
import logging
import json

In [3]:
class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{token}",flush=True,end="")

In [19]:
chat_llm_model = "gpt-3.5-turbo-1106"
qa_llm_model = "gpt-3.5-turbo"
children_chunk_size=1000
children_chunk_overlap=100
chunk_overlap = 200
chunk_size = 10000
parent_k = 20
children_k = 60
RESOURCE_FOLDER_PATH="C:\\Users\\ChandPashaShaik\\Downloads\\resumes 1\\resumes"
RESOURCE_FOLDER_PATH="C:\\Users\\ChandPashaShaik\\Downloads\\parsed_resumes_json\\parsed_resumes_json"

In [44]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=children_chunk_size,chunk_overlap=children_chunk_overlap)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)

llm = ChatOpenAI(temperature=0.1,streaming=False,model=chat_llm_model)
streaming_llm = ChatOpenAI(temperature=0.1,streaming=True,model=chat_llm_model)
embeddings = OpenAIEmbeddings()



In [6]:
def get_ic_document_paths():
    ic_documents_paths = glob(pathname=RESOURCE_FOLDER_PATH+"**/*.docx",recursive=True)+glob(pathname=RESOURCE_FOLDER_PATH+"**/*.pdf",recursive=True)+glob(pathname=RESOURCE_FOLDER_PATH+"**/*.txt",recursive=True)
    # ic_document_names = [os.path.basename(ic_documents_path) for ic_documents_path in ic_documents_paths]
    return ic_documents_paths
    
def get_ic_document_names():
    ic_documents_paths = glob(pathname=RESOURCE_FOLDER_PATH+"**/*.docx",recursive=True)+glob(pathname=RESOURCE_FOLDER_PATH+"**/*.pdf",recursive=True)+glob(pathname=RESOURCE_FOLDER_PATH+"**/*.txt",recursive=True)
    ic_document_names = [os.path.basename(ic_documents_path) for ic_documents_path in ic_documents_paths]
    return ic_document_names

In [7]:
%%time
ic_documents_paths = get_ic_document_paths()

[print(item) for item in ic_documents_paths]

docs = []

for ic_documents_path in ic_documents_paths:
    
    temp_docs = UnstructuredFileLoader(file_path=ic_documents_path,mode="single").load()


    for temp_doc in temp_docs:
        #temp_doc.metadata["doc_id"] = str(uuid.uuid4())
        temp_doc.metadata['source'] = os.path.basename(ic_documents_path)
        temp_doc.metadata['file_path'] = ic_documents_path
        splitted_docs = parent_splitter.split_documents([temp_doc])
        for splitted_doc in splitted_docs:
            splitted_doc.metadata["doc_id"] = str(uuid.uuid4())
            # splitted_doc.metadata['parent'] = splitted_doc
        docs.extend(splitted_docs)
    


min_ln = len(docs[0].page_content)
max_ln = 0
for doc in docs:
    if len(doc.page_content)<min_ln:
        min_ln = len(doc.page_content)
    if len(doc.page_content) > max_ln:
        max_ln = len(doc.page_content)


print(f"character lengths: {min_ln=};{max_ln=}")

C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Alex Pappalardi.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Amy Ahern Resume.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Amy Halter Resume[39].txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\April 2023 POA Agenda.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Beck, Brad .txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Blake Mitchell Resume.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Bochiechio_Dominic_Resume 1.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Brad Westcott Resume 2023 v2.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\Bradley W.txt
C:\Users\ChandPashaShaik\Downloads\parsed_resumes_json\parsed_resumes_json\cael-saxton-cv.txt
C:\Users\ChandPas

C:\Users\ChandPashaShaik\anaconda3\envs\file_analyst_ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


character lengths: min_ln=45;max_ln=5209
CPU times: total: 5.47 s
Wall time: 14.8 s


In [8]:
children_chunks = []
children_chunks = child_splitter.split_documents(docs)
min_ln = len(children_chunks[0].page_content)
max_ln = 0
for doc in children_chunks:
    if len(doc.page_content)<min_ln:
        min_ln = len(doc.page_content)
    if len(doc.page_content) > max_ln:
        max_ln = len(doc.page_content)


print(f"character lengths: {min_ln=};{max_ln=}")

character lengths: min_ln=45;max_ln=1000


In [9]:
get_ic_document_names()

['Alex Pappalardi.txt',
 'Amy Ahern Resume.txt',
 'Amy Halter Resume[39].txt',
 'April 2023 POA Agenda.txt',
 'Beck, Brad .txt',
 'Blake Mitchell Resume.txt',
 'Bochiechio_Dominic_Resume 1.txt',
 'Brad Westcott Resume 2023 v2.txt',
 'Bradley W.txt',
 'cael-saxton-cv.txt',
 'Carelle Jonassaint PRI.txt',
 'Charles Nardi_Resume.txt',
 'Charles Nardi_Scorecard.txt',
 'Cheryl Provorny - Resume (3).txt',
 'Chris Harber_TimeDoc CEO.txt',
 'Chris Jones_Resume.txt',
 'Chris Jones_Scorecard.txt',
 'Chris Lobdell - ProcessMaker CEO Scorecard.txt',
 'Chris Lobdell Resume 11212022.txt',
 'christina-oelhafen-cv.txt',
 'Christine Aiello PRI.txt',
 'Connor Fu - Resume (3).txt',
 'Craig Jones_Scorecard.txt',
 'David Coppeans_TimeDoc CEO.txt',
 'David Driscoll Resume.txt',
 'Dean Sawyer Digital Health CEO 1.txt',
 'deep-gopani-cv (1).txt',
 'Dennis Ratzker.txt',
 'DeWitt Bio Jan 2023.txt',
 'Doug Johnson_Resume.txt',
 'Doug Johnson_Scorecard.txt',
 'Dustin Sapp_Resume.txt',
 'Dustin Sapp_Scorecard.txt',

In [10]:
db = FAISS.from_documents(documents=children_chunks,embedding=embeddings)

db.save_local(folder_path=RESOURCE_FOLDER_PATH+"\\vector_database",index_name="resume_json")

db.index.ntotal

# db = FAISS.load_local(folder_path=RESOURCE_FOLDER_PATH+"\\vector_database",index_name="ic_documents",embeddings=embeddings)

358

### ask openai whether he is asking for perticular files?

In [39]:
FILTER_FILE_NAME_PROMPT = """

As a highly skilled NLP bot, your primary role involves analyzing user prompts that come with file names. Your task is to discern the essence of the prompt and, if necessary, rephrase it while identifying relevant file names. This is especially important when a specific question about a file is posed.


%EXAMPLES:
---------------------------------------
1. summarize the documents from 10-10-2023 to 11-30-2023
%Files:
The collector 10-10-2023.pdf
Mad max 30-11-2023.pdf
Movie Maker 20-11-2023

%Answer:
{{
"rephrased_prompt":"summerize",
"file_names":["The collector 10-10-2023.pdf","Mad max 30-11-2023.pdf"]
}}

+=================================================+
2. summarize the documents from 12-10-2023 to 11-30-2024
%Files:
The collector 10-10-2023.pdf
Mad max 30-11-2023.pdf
Movie Maker 20-11-2023

%Answer:
{{}}
=================================================================
3. wha is summary of collector file?
%Files:
The collector 10-10-2023.pdf
Mad max 30-11-2023.pdf
Movie Maker 20-11-2023

%Answer:
{{
"rephrased_prompt":"summerize",
"file_names":["The collector 10-10-2023.pdf"]
}}

=================================================================
3. who is the author?
%Files:
The collector 10-10-2023.pdf
Mad max 30-11-2023.pdf
Movie Maker 20-11-2023

%Answer:
{{}}
=================================================================

%% Question:
{question}

%%% File Names:
{file_names}

%%%INSTRUCTIONS:
{format_instructions}

"""



response_schemas = [
    ResponseSchema(name="file_names", description="relavant file names",type="list"),
    ResponseSchema(name="rephrased_prompt", description="rephrased prompt",type="string")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

FILTER_FILE_NAME_PROMPT_TEMPLATE = PromptTemplate(template=FILTER_FILE_NAME_PROMPT,input_variables=["question","file_names"],partial_variables={"format_instructions": format_instructions},output_parser=output_parser)


filter_file_chain = LLMChain(llm=OpenAI(),prompt=FILTER_FILE_NAME_PROMPT_TEMPLATE)

def get_filter_files_names(prompt:str):
    filtered_file_names = filter_file_chain.run({"question":prompt,"file_names":get_ic_document_names()})
    # print(f"{filtered_file_names=}")
    try:
        filtered_file_names= output_parser.parse(filtered_file_names)["file_names"]
        if filtered_file_names and isinstance(filtered_file_names,list):
            return {
                "source":filtered_file_names
            }
    except Exception as error:
        # print(error)
        pass


In [ ]:
# get_filter_files_names("how many companies are there?")

In [40]:


redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76,k=parent_k)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter]
)

def get_relavant_chunks(prompt:str):
    retriever = db.as_retriever(search_kwargs=dict(filter=get_filter_files_names(prompt),k=children_k,fetch_k=100))
    compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)
    chunks = compression_retriever.get_relevant_documents(prompt)
    if len(chunks)==0:
        return retriever.get_relevant_documents(prompt)[:10]
    return chunks


In [41]:
def get_relavant_documents(prompt:str):
    relavant_chunks = get_relavant_chunks(prompt)
    print(f"{len(relavant_chunks)=}")
    if len(relavant_chunks)==0:
        return []
    parent_ids = list({chunk.metadata["doc_id"]:chunk for chunk in relavant_chunks})

        
    # least_relavant = min([x.state['query_similarity_score'] for x in relavant_chunks])
    print(f"Unique parent documents size {len(parent_ids)}")
    # print(f"{least_relavant=}")
    # print([relavant_chunk.state['query_similarity_score'] for relavant_chunk in relavant_chunks])
    parent_documents = []
    for parent_id in parent_ids:
        l = list(filter(lambda x: x.metadata["doc_id"]==parent_id,docs))
        if len(l)>0:
            parent_documents.append(l[0])
    # parent_documents = [ list(filter(lambda x: x.metadata["doc_id"]==parent_id,docs))[0] for parent_id in parent_ids]
    return parent_documents

In [ ]:
# relavant_docs = get_relavant_documents("how many companies are in Interest Checks 9.11.23 - 9.22.23.docx")
# len(relavant_docs)

In [49]:


# qa_chain = load_qa_chain(llm=ChatOpenAI(model=chat_llm_model,temperature=0.1),chain_type="map_reduce",verbose=False,reduce_llm=streaming_llm,token_max=7000)
# qa_chain = load_qa_chain(llm=streaming_llm,chain_type="stuff",verbose=False)

async def get_response(prompt:str):
    
    qa_chain = load_qa_chain(llm=streaming_llm,chain_type="stuff",verbose=False)
    relavant_chunks = get_relavant_documents(prompt)
    lengths = [len(doc.page_content) for doc in relavant_chunks]
    print(f"sum(lengths)={sum(lengths)}")
    if len(lengths)>0 and sum(lengths)>30_000:
        qa_chain = load_qa_chain(llm=ChatOpenAI(model=chat_llm_model,temperature=0.1),chain_type="map_reduce",verbose=False,reduce_llm=streaming_llm,token_max=7000)
    # print(len(lengths))
    print(f"{prompt=}")
    print("=========================================")
    print("Response")
    print("=========================================")
    response = None
    with get_openai_callback() as cb:
        response =  await qa_chain.arun(question=prompt,input_documents=relavant_chunks,callbacks=[MyCustomHandler()])
        print(f"\n\nTotal cost for this chat = ${cb.total_cost}")
    # print(response)
    return response,relavant_chunks

In [51]:
user_prompt = """
comprehensive summary

"""
response,_ = await get_response(user_prompt)

len(relavant_chunks)=20
Unique parent documents size 19
sum(lengths)=46787
prompt='\ncomprehensive summary\n\n'
Response
I'm sorry, but I cannot provide a comprehensive summary of the document as it contains personal information. However, I can help with specific questions or tasks related to the content. If you have a specific question or need information on a particular topic from the document, please feel free to ask.

Total cost for this chat = $0.015863


In [ ]:
Markdown(response["output_text"])

In [ ]:
[len(__.page_content) for __ in _]